### Task 5

Проведём подобие **А/В тестирования** \
Тестирования для баз данных `test` - с учащимися, что посещали ленту новостей и `control` - с теми, которые её не просматривали \
Для этого вычислим два средних значения `delta` до первого посещения ленты новостей и после, для тестовой и контрольной групп, т.к. учащиеся контрольной группы не посещали ленту новостей, то мы будем использовать среднее значение даты первого посещения в таблице `test`, ранее мы его вычислили и сохранили в таблице `control`

1. Используя библиотеку *sqlite3*, создадим соединение с базой данных

In [ ]:
import sqlite3

db_con = sqlite3.connect('..\..\data\checking-logs.sqlite')

2. Используя данные таблиц `test` и `control` и только *один* запрос создадим два датафрейма **test_results** и **control_results**:
    - новые датафреймы содержат 2 столбца `times` и `avg_diff`
    - новые датафреймы содержат 2 строки, соответственно столбец `times` имет значения `before` и `after`
    - столбец `avg_diff` содержит среднее значение `delta`, учитывающее всех учащихся за период времени до первого посещения ленты новостей и после, соответственно
    - в анализе учавствуют учащиеся у которых есть значения и `before`, и `after`
    - задание `project1` не учитываем 

In [2]:
import pandas as pd

test_results = pd.io.sql.read_sql('''SELECT times, AVG(avg_diff) as avg_diff
        FROM (
            SELECT uid, labname,
                AVG ((strftime("%s", first_commit_ts) - deadlines) / 3600) AS avg_diff,
                    CASE
                        WHEN (strftime("%s", first_commit_ts)) < (strftime("%s", first_view_ts))
                        THEN "before" ELSE "after" END AS times
            FROM test
            LEFT JOIN deadlines ON test.labname = deadlines.labs
            WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")
            GROUP BY uid, times)
        WHERE uid IN (
            SELECT uid
            FROM (
                SELECT uid, 
                    CASE
                        WHEN (strftime("%s", first_commit_ts)) < (strftime("%s", first_view_ts))
                        THEN "before" ELSE "after" END AS times
                FROM test
                LEFT JOIN deadlines ON test.labname = deadlines.labs
                WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")
                GROUP BY uid, times)
        GROUP BY uid
        HAVING COUNT(uid) == 2)
        GROUP BY times''', db_con)
test_results

,times,avg_diff
0,after,-99.523810
1,before,-66.047619


In [3]:
control_results = pd.io.sql.read_sql('''SELECT times, AVG(avg_diff) as avg_diff
        FROM (
            SELECT uid, labname,
                AVG ((strftime("%s", first_commit_ts) - deadlines) / 3600) AS avg_diff,
                    CASE
                        WHEN (strftime("%s", first_commit_ts)) < (strftime("%s", first_view_ts))
                        THEN "before" ELSE "after" END AS times
            FROM control
            LEFT JOIN deadlines ON control.labname = deadlines.labs
            WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")
            GROUP BY uid, times)
        WHERE uid IN (
            SELECT uid
            FROM (
                SELECT uid, 
                    CASE
                        WHEN (strftime("%s", first_commit_ts)) < (strftime("%s", first_view_ts))
                        THEN "before" ELSE "after" END AS times
                FROM control
                LEFT JOIN deadlines ON control.labname = deadlines.labs
                WHERE labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s")
                GROUP BY uid, times)
        GROUP BY uid
        HAVING COUNT(uid) == 2)
        GROUP BY times''', db_con)
control_results

,times,avg_diff
0,after,-99.322222
1,before,-98.033333


3. Закроим соединение с базой данных

In [4]:
db_con.close()

4. Наконец, можем сделать вывод, повлияло ли наличие ленты новостей на мотивацию учащихся \
Если значение `delta` в тестовой группе значительно отличается перед первым посещением учащимися ленты новостей и после посещения и подобного не происходит в контрольной группе - значит мы можем рекомендовать компании распространить идею создания ленты новостей на платформе для всех обучающихся

Предположение верное, посещение ленты новостей влияет на поведение учащихся - сроки сдачи в тестовой группе улучшились.